## Implement Batch Gradient Descent with early stopping for Softmax Regression (without using Scikit-Learn).

In [1]:
from sklearn import datasets
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
def one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    y_one_hot = np.zeros((m,n_classes))
    y_one_hot[np.arange(m), y] = 1
    return y_one_hot

def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

## Data Preprocessing

In [3]:
# Load iris dataset
iris = datasets.load_iris()
X = iris["data"][:, (2,3)] # petal length & width
y = iris["target"]

# Adding the bias term for every instance (Xo = 1)
X_with_bias = np.c_[np.ones([len(X), 1]), X]

# Setting the random seed for reproducibility
np.random.seed(2042)

In [4]:
# Splitting training and test sets
total_size = len(X_with_bias)
split_ratio = 0.2

test_size, validation_size = int(total_size * split_ratio), int(total_size * split_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [5]:
# One-hot encoding
y_train_one_hot = one_hot(y_train)
y_valid_one_hot = one_hot(y_valid)
y_test_one_hot = one_hot(y_test)

In [6]:
n_inputs = X_train.shape[1] # petal length, petal width, bias
n_outputs = len(np.unique(y_train)) # number of classes (3)

### Softmax Function

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$



### Cost Function
$J(\mathbf{\Theta}) = \dfrac{1}{m}\sum\limits{i=1}^{m}\sum\limits{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

### And the equation for the gradients:
$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

In [7]:
eta = 0.01 # learning rate
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

# Random initialization
theta = np.random.randn(n_inputs, n_outputs)

# Training a softmax model
for iteration in range(n_iterations):
    logits = X_train.dot(theta)
    y_proba = softmax(logits)
    loss = -np.mean(np.sum(y_train_one_hot * np.log(y_proba + epsilon), axis=1))
    error = y_proba - y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = 1/m * X_train.T.dot(error)
    theta = theta - eta * gradients

0 5.446205811872683
500 0.8350062641405651
1000 0.6878801447192402
1500 0.6012379137693313
2000 0.5444496861981873
2500 0.5038530181431525
3000 0.4729228972192248
3500 0.4482424418895776
4000 0.4278651093928793
4500 0.41060071429187134
5000 0.3956780375390373


In [8]:
# Predictions on the validation data
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis = 1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667

#### Regularized model with increased learning rate

In [9]:
eta = 0.1
alpha = 0.1  # regularization hyperparameter

# Random initialization
theta = np.random.randn(n_inputs, n_outputs)

# Training a softmax model
for iteration in range(n_iterations):
    logits = X_train.dot(theta)
    y_proba = softmax(logits)
    entropy_loss = -np.mean(np.sum(y_train_one_hot * np.log(y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(theta[1:]))
    loss = entropy_loss + alpha * l2_loss
    error = y_proba - y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * theta[1:]]
    theta = theta - eta * gradients

0 6.629842469083912
500 0.5339667976629506
1000 0.503640075014894
1500 0.49468910594603216
2000 0.4912968418075477
2500 0.48989924700933296
3000 0.48929905984511984
3500 0.48903512443978603
4000 0.4889173621830818
4500 0.4888643337449303
5000 0.4888403120738818


In [10]:
# Predictions on the validation data
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis = 1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

### Adding early stopping

##### Measure the loss on the validation set at every iteration and stop when the error starts growing.

In [11]:
best_loss = np.infty

# Random initialization
theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    # For training set
    logits = X_train.dot(theta)
    y_proba = softmax(logits)
    entropy_loss = -np.mean(np.sum(y_train_one_hot * np.log(y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(theta[1:]))
    loss = entropy_loss + alpha * l2_loss
    error = y_proba - y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * theta[1:]]
    theta = theta - eta * gradients
    
    # For test set
    logits = X_valid.dot(theta)
    y_proba = softmax(logits)
    entropy_loss = -np.mean(np.sum(y_valid_one_hot * np.log(y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(theta[1:]))
    loss = entropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "Early Stopping!")
        break

0 4.7096017363419875
500 0.5739711987633519
1000 0.5435638529109127
1500 0.5355752782580262
2000 0.5331959249285544
2500 0.5325946767399383
2765 0.5325460966791898
2766 0.5325460971327975 Early Stopping!


In [12]:
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis = 1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

##### Measure the final model's accuracy on test set:

In [13]:
logits = X_test.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

acc_score = np.mean(y_predict == y_test)
acc_score

0.9333333333333333